In [0]:
# FUNÇÕES UTILIZADAS => (FILTROS) =>  (select) / (filter=where) / (like) / (groupBy) / (orderBy) / (drop)

# LINK YOUTUBE => https://www.youtube.com/watch?v=MOQPqYP-0Rg&list=PLOqx8yEV2-l2Oce3vUNR7L7Qjx_sy8-WO&index=1&t=163s

In [0]:
# IMPORTANDO BIBLIOTECAS PARA TRABALHAR COM (SPARK)
import pyspark.sql.functions 
from pyspark.sql.types import StructType, StructField , StringType ,IntegerType
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# CRIANDO SESSÃO DO SPARK 

spark = (SparkSession.builder
        .master("local(*)")
        .appName('video-1')
        .getOrCreate()
        )

In [0]:
# CRIANDO DADOS  ==> DATAFRAME CARRO 

dados = (

    ['1','Celta','Chevrolet',2004,18000,"Não"],
    ['2','Gol','volkswagen',2020,46320,"Sim"],
    ['3','Corsa Classic', 'Chevrolet',2008,15000,"Sim"],
    ['4','Palio','Fiat',1998,8900,"Não"],
    ['5','ASX','Mitsubishi',2016,65000,"Sim"],
    ['6','Corolla','Toyota',2018,89990,"Sim"],
    ['7','Civic','Honda',2019,89990,"Sim"],
    ['8','Eco Sport','Ford',2017,69299,"Sim"],
    ['9','Onix','Chevrolet',2020,58990,"Sim"]

)

# SCHEMA ==> COLUNAS 

colunas = (StructType([
    StructField('cod' , StringType(), True),
    StructField('Carro' , StringType(), True),
    StructField('Marca' , StringType(), True),
    StructField('Ano' , IntegerType(), True),
    StructField('Preco' , IntegerType(), True),
    StructField('Flex' , StringType(), True)
]))

# DATAFRAME FINAL 

df = spark.createDataFrame(data = dados , schema = colunas)

df.display()

# VERIFICANDO O (SCHEMA)
print('\nSchema da tabela\n')
df.printSchema()

cod,Carro,Marca,Ano,Preco,Flex
1,Celta,Chevrolet,2004,18000,Não
2,Gol,volkswagen,2020,46320,Sim
3,Corsa Classic,Chevrolet,2008,15000,Sim
4,Palio,Fiat,1998,8900,Não
5,ASX,Mitsubishi,2016,65000,Sim
6,Corolla,Toyota,2018,89990,Sim
7,Civic,Honda,2019,89990,Sim
8,Eco Sport,Ford,2017,69299,Sim
9,Onix,Chevrolet,2020,58990,Sim



Schema da tabela

root
 |-- cod: string (nullable = true)
 |-- Carro: string (nullable = true)
 |-- Marca: string (nullable = true)
 |-- Ano: integer (nullable = true)
 |-- Preco: integer (nullable = true)
 |-- Flex: string (nullable = true)



In [0]:
# 1) Encontre carros que comecem com a letra "A" 

df.filter(col('Carro').like('A%')).display()

#df.filter(df.Carro.like('A%')).display()

#df.filter("carro like 'A%'").display()

# SELECIONANDO APENAS A COLUNA (Carros)
#df.select(col('Carro')).filter(col('Carro').like('A%')).display()


# MESMO EM SQL 

#df.createOrReplaceTempView("carro")

#teste = spark.sql("Select Carro from carro WHERE Carro LIKE 'A%' ").show()

cod,Carro,Marca,Ano,Preco,Flex
5,ASX,Mitsubishi,2016,65000,Sim


In [0]:
# 2) SELECIONE AS MARCAS QUE COMECEM COM ("C")

df.filter(col('Marca').like('C%')).display()

cod,Carro,Marca,Ano,Preco,Flex
1,Celta,Chevrolet,2004,18000,Não
3,Corsa Classic,Chevrolet,2008,15000,Sim
9,Onix,Chevrolet,2020,58990,Sim


In [0]:
# 3) TRAGA OS CARROS QUE SÃO (FLEX) E A MARCA COMECE COM A LETRA ("v") ==> (& == AND) ; (ou ; or = |)

#df.filter(col('Flex') == 'Sim').filter(col('Marca').like('v%')).display()

df.where( (col('flex') == 'Sim')  &  (col('Marca').like('v%')) ).display()

cod,Carro,Marca,Ano,Preco,Flex
2,Gol,volkswagen,2020,46320,Sim


In [0]:
# 4) TRAGA CARROS QUE (NÃO SÃO FLEX) COM A LETRA FINAL ("a")


df.filter(col('Flex') == 'Não').filter(col('Carro').like('%a')).display()

# df.filter( (col('Flex') == 'Não') & (col('Carro').like('%a')) ).display()

cod,Carro,Marca,Ano,Preco,Flex
1,Celta,Chevrolet,2004,18000,Não


In [0]:
# 5) ENCONTRE O CARRO QUE COMECE COM A LETRA ("C") (NÃO SEJA FLEX) E SEJA DO (ANO = 2004)

#df.where((col('Carro').like('C%') & (col('Flex') == "Não") & (col('Ano') == 2004))).display()

#df.filter(col('Carro').like('C%')).filter(col('Flex') == "Não").filter(col('Ano') == 2004).display()

df.where(col('Carro').like('C%')).where(col('Flex') == "Não").where(col('Ano') == 2004).display()

cod,Carro,Marca,Ano,Preco,Flex
1,Celta,Chevrolet,2004,18000,Não


In [0]:
# 6) TRAGA CARROS QUE COMECEM COM A LETRA (T) E (P) 

# ERRO PROPOSITAL ABAIXO==> PQ SE ELE NAO ACHAR O PRIMEIRO (FILTRO) ELE JA PARA ==> ENTAO PRECISO USAR (WHERE) COM OPERADORES LOGICOS 

#df.filter(col('Carro').like('T%')).filter(col('Carro').like('P%')).display()

# AQUI ELE VAI ACHAR OU UM OUTRO ==> (| = OR )

df.where((col('Carro').like('T%') | (col('Carro').like('P%')))).display() # no caso só VAI ACHAR O ("P") ==> (t) não tem


cod,Carro,Marca,Ano,Preco,Flex
4,Palio,Fiat,1998,8900,Não


In [0]:
# 7) TRAGA CARROS QUE COMECEM COM A LETRA (C) E TENHAM (ANO) IGUAL A 2008  E PODEM SER (FLEX) (OU) NÃO 

##==> AQUI N VAI ACHAR NADA, PQ É FLEX
#df.filter(col('Carro').startswith('C')).where(col('Ano') == 2008).where(col('Flex') == 'Não').display() 


# USANDO OPERADORES ==> (AND = &) e (OR = |)

df.where((col('Carro').startswith('C') & (col('Ano') == 2008) | (col('Flex') == "Não" ))).display()

cod,Carro,Marca,Ano,Preco,Flex
1,Celta,Chevrolet,2004,18000,Não
3,Corsa Classic,Chevrolet,2008,15000,Sim
4,Palio,Fiat,1998,8900,Não


# UTILIZANDO (groupBy) / (orderBy = sort) / (drop)

In [0]:
# OBS => (groupBy)==> só é utilizado com (AGG) => count / sum / avg / min / max

In [0]:
# 8) TRAGA TODAS AS (MARCAS) E AS SUAS QUANTIDADES 

#df.groupBy('Marca').count().display()

df.groupBy('Marca').agg(count('Marca').alias('Qnt_marcas')).display()

Marca,Qnt_marcas
Chevrolet,3
volkswagen,1
Fiat,1
Mitsubishi,1
Toyota,1
Honda,1
Ford,1


In [0]:
# 9) Ordene a coluna (Ano) do menor ano até o maior   ==> OBS ==> orderBy = sort

df.orderBy(col('Ano').asc()).display()

#df.orderBy('ano').display()

cod,Carro,Marca,Ano,Preco,Flex
4,Palio,Fiat,1998,8900,Não
1,Celta,Chevrolet,2004,18000,Não
3,Corsa Classic,Chevrolet,2008,15000,Sim
5,ASX,Mitsubishi,2016,65000,Sim
8,Eco Sport,Ford,2017,69299,Sim
6,Corolla,Toyota,2018,89990,Sim
7,Civic,Honda,2019,89990,Sim
9,Onix,Chevrolet,2020,58990,Sim
2,Gol,volkswagen,2020,46320,Sim


In [0]:
# 10) Escolha todas as (marcas) (UNICAS) e ordene a (QUANTIDADE) (DA MAIOR PARA MENOR) 

df.groupBy(col('Marca')).count().orderBy(col('count').desc()).display()

Marca,count
Chevrolet,3
volkswagen,1
Mitsubishi,1
Fiat,1
Toyota,1
Honda,1
Ford,1


In [0]:
# 11) APAGUE A COLUNA (FLEX) e (COD)

df.drop('Flex','cod').display()

Carro,Marca,Ano,Preco
Celta,Chevrolet,2004,18000
Gol,volkswagen,2020,46320
Corsa Classic,Chevrolet,2008,15000
Palio,Fiat,1998,8900
ASX,Mitsubishi,2016,65000
Corolla,Toyota,2018,89990
Civic,Honda,2019,89990
Eco Sport,Ford,2017,69299
Onix,Chevrolet,2020,58990


In [0]:
# 2) video será sobre (isin) ==> filtro mais aprofundado que (filter/where)